# Compte-rendu - projet de génération d'un modèle de détection de mouvement de balancier

## TOC
* [Partie 1](#1)
* [Partie 2](#2)
* [Partie 3](#3)
* [Partie 4](#4)
* [Partie 5](#5)
* [Partie 6](#6)
* [Partie 7](#7)
* [Partie 8](#8)
* [Partie 9](#9)
* [Partie 10](#10)

## Initialisation <a class="anchor" id="1"></a>
Import des packages utile

* `pandas` : librairie pour manipuler les données (`DataFrame`)
* `numpy` : librairie mathématique
* `matplotlib` : librairie graphique

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Récupération des données <a class="anchor" id="2"></a>

Nous avons au préalable réalisé des enregistrements de données des capteurs (disponibles dans le dossier `/data`).
Nous avons stocké dans un [Google sheet](https://docs.google.com/spreadsheets/d/1By59dQ56zL_kP0tW9Iyf4FppyEvJtcwnuG4gx1iokpM/edit?usp=sharing) si les captures correspondent à un état de balancier, ou non.
Nous devons traiter cette donnée brute pour la rendre plus compréhensible et interprétable par Keras.

La première étape est de trim les enregistrements pour ne garder que la partie qui nous intéresse.
Cela est surtout important dans le cas où notre capture représente un mouvement de balancier, car la donnée brute inclut, au début et à la fin, des instants où l'objet ne se balance pas. Cela correspond au temps qu'il s'écoule entre l'activation de la capture et le début du mouvement.

In [ ]:
# Limites des données utiles de chaque fichier .csv
LIMITES = [ [60, 650],   [50, 350],  [100, 600],
            [0, 550],    [0, 550],   [150, 250],
            [0, 700],    [75, 820],  [0, 500],
            [50, 550],   [50, 750],  [50, 900],
            [0, 900],    [50, 1000], [100, 1200],
            [50, 1300],  [50, 900],  [200, 700],
            [0, 60],     [0, 60],    [0, 25],
            [100, 800],  [100, 900], [100, 600],
            [100, 850],  [50, 900],  [100, 1500],
            [100, 2200], [30, 300],  [0, 650],
            [0, 550],    [100, 500], [0, 710]]

len(LIMITES)

In [ ]:
dataset =[]

def trimDataset(ds):
    for i in range(len(ds)):
        ds[i] = ds[i][LIMITES[i][0]:LIMITES[i][1]]


for i in range(len(LIMITES)):
    dataset.append(pd.read_csv("data/balancier" + str(i) + ".csv"))
    trimDataset(dataset)

len(dataset)

_________________________________________________________

# Analyse des données
On commence par importer les données depuis un fichier CSV

In [ ]:
d = pd.read_csv('data/old/SensorTile_Log_N008.csv')
d.columns

In [ ]:
def cleanDataframe(df):
    df = df.drop(columns=["T [ms]"])
    return df

def sliceDf(df, step):
    res = []
    while (len(df) > step):
        res.append(df.iloc[:step])
        df = df.iloc[step:]
    return res

In [ ]:
#d = cleanDataframe(d)
d.tail(10)

In [ ]:
dfs = sliceDf(d, 100)
len(d)

In [ ]:
len(dfs)

In [ ]:
plt.plot(d["AccX [mg]"])
plt.plot(d["AccY [mg]"])
plt.plot(d["AccZ [mg]"])

In [ ]:
plt.plot(d["GyroX [mdps]"])

# Fetch balancier data depuis Gsheet

Récupération du résultat attendu depuis un Google Sheet

In [ ]:
from gSheet import SheetAPI

In [ ]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1By59dQ56zL_kP0tW9Iyf4FppyEvJtcwnuG4gx1iokpM'
api = SheetAPI(SPREADSHEET_ID)
api.connect()
print(api.getValues("A2:D100"))

# Parsing the data

Given the data, we can parse it to extract the information we need.

First we slice the dataframe into multiple 1-sec **rolling** windows
Then we multiply the data by sampling the data into subsets.

In [ ]:
SAMPLE_RATE = 50 # Hz
DATA_RATE = 20 # Hz
WINDOW_LENGTH = 2000 # (in ms)

In [ ]:

def sliceDf(df, step):
    res = []
    while (len(df) > step):
        res.append(df.iloc[:step])
        df = df.iloc[1:]
    return res

def removeTime(df):
    df = df.drop(columns=["T [ms]"])
    return df

def sampleDf(df, sample):
    res = []
    for i in range(0, sample): # A tester
        res.append(df.iloc[lambda x: x.index % sample == i])
    return res

In [ ]:
d = pd.read_csv('data/balancier0.csv')
d.columns

In [ ]:
d.tail(10)